In [1]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
from math import radians
from io import StringIO



##### Test 1

In [2]:
# https://stackoverflow.com/questions/56862277/interpreting-sklearn-haversine-outputs-to-kilometers

earth_radius = 6371000 # meters in earth
test_radius = 1300000 # meters

test_points = [[32.027240,-81.093190],[41.981876,-87.969982]]
test_points_rad = np.array([[radians(x[0]), radians(x[1])] for x in test_points ])

tree = BallTree(test_points_rad, metric = 'haversine')
ind,results = tree.query_radius(test_points_rad, r=test_radius/earth_radius) #return_distance  = True
# print(ind)
print(results * earth_radius/1000)

[   0. 6371.]


##### Test 2

In [3]:
# https://stackoverflow.com/questions/61952561/how-do-i-find-the-neighbors-of-points-containing-coordinates-in-python

cities = [
    ['B', 50.94029883, 7.019146728], 
    ['C', 50.92073002, 6.975268711], 
    ['D', 50.99807758, 6.980865543],
    ['E', 50.98074288, 7.035060206],
    ['F', 51.00696972, 7.035993783],
    ['G', 50.97369889, 6.928538763],
    ['H', 50.94133859, 6.927878587],
    ['A', 50.96712502, 6.977825322]
]

cities_2=[
    [1, 'The Nairobi West Hospital-Utawala', 36.899077, -1.299067], 
    [4, 'Medpearl HealthCare Services Limited', 36.8368824, -1.3114241], 
    [6, 'St Monicah Dispensary Mwengenye', 36.9187518, -1.2521117],
    [10, 'Inland Medical Center', 36.8947267, -1.2660535],
    [22, 'The Nairobi dental suite', 36.774204, -1.289479],
    [1009, 'Kimandi Superlife Medical Clinic', 36.8309148, -0.8117988],
    [4217, 'Ruma Ubuntu Afya Clinic', 34.335239, -0.23085],
    [4246, 'Kirathimo Medical Clinic Makuyu', 37.2775217, -0.9055267],
    [6627, 'Kapteldet Dispensary', 35.21737, 0.45398],
    [10877, 'Kianjege Dispensary', 37.16596, -0.574],
]

cities_np = np.array(cities_2)

df = pd.DataFrame(cities_np, columns=['id', 'City', 'Latitude', 'Longitude'])

# raise

# print(df[['Latitude', 'Longitude']].values.astype(float))

# print(df)

tree = BallTree(np.deg2rad(df[['Latitude', 'Longitude']].values.astype(float)), metric='haversine')

# Setup distance queries (points for which we want to find nearest neighbors)
other_data = """NAME Latitude Longitude
B_alt 50.94029883 7.019146728
C_alt 50.92073002 6.975268711"""

other_data = """id NAME Latitude Longitude
0 B_alt 36.899077 -1.299067"""

df_other = pd.read_csv(StringIO(other_data), sep = ' ')

query_lats = df_other['Latitude']
query_lons = df_other['Longitude']

print(df_other)


distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 3)

r_km = 6371 # multiplier to convert to km (from unit distance)

print(distances, indices)

for name, d, ind in zip(df_other['NAME'], distances, indices):
    # print(f"NAME {name} closest matches:")
    # print(name, d, ind)
    for i, index in enumerate(ind):
        print(i, index)
        print(f"\t{df['City'][index]} with distance {d[i]*r_km:.4f} km")


   id   NAME   Latitude  Longitude
0   0  B_alt  36.899077  -1.299067
[[0.         0.00046701 0.00073981]] [[0 3 2]]
0 0
	The Nairobi West Hospital-Utawala with distance 0.0000 km
1 3
	Inland Medical Center with distance 2.9753 km
2 2
	St Monicah Dispensary Mwengenye with distance 4.7133 km


### Implementation

In [16]:
# https://pythontic.com/pandas/serialization/postgresql#:~:text=Data%20from%20a%20PostgreSQL%20table,SQLAlchemy%20Engine%20as%20a%20parameter.

from sqlalchemy import create_engine

engine = create_engine("postgresql+psycopg2:///?user=postgres&password=123@Team&host=127.0.0.1&port=5432&database=kmfl")
# dbConnection = engine.connect()

df2 = pd.read_sql("""select 
    name, officialname, keph_level_name, facility_type_name, county_name, 
	sub_county_name, owner_type_name, admission_status_name, 
	operation_status_name, 
	open_whole_day, open_public_holidays, open_weekends, open_late_night,
	service_names,
    lat,long, 
    '' as distance 
    from tbl_kmfl 
    where lat is not null
    
""", con=engine)

In [17]:
df2

,name,officialname,keph_level_name,facility_type_name,county_name,sub_county_name,owner_type_name,admission_status_name,operation_status_name,open_whole_day,open_public_holidays,open_weekends,open_late_night,service_names,lat,long,distance
0,Voi Prison VCT,Voi Prison VCT,Level 2,VCT,Taita Taveta,Voi,Ministry of Health,Not Admitting Patients,Operational,False,False,False,False,"['Condom Distribution & STI Prevention', 'HIV ...",-3.389984,38.560666,
1,Al Shafic Nursing Home Limited,Al Shafic Nursing Home Limited,Level 3,Nursing Homes,Wajir,Wajir East,Private Practice,Admitting General Patients Only,Operational,True,True,True,True,"['Accident and Emergency casualty Services', '...",1.744050,40.044888,
2,Medpearl HealthCare Services Limited,Medpearl HealthCare Services Limited,Level 3,Medical Center,Nairobi,Starehe,Private Practice,Not Admitting Patients,Operational,False,False,False,False,"['Basic dental services', 'Focused Antenatal C...",-1.311424,36.836882,
3,Kaptiien Dispensary,Kaptien Dispensary,Level 2,Dispensary,Trans Nzoia,Kiminini,Ministry of Health,Not Admitting Patients,Operational,False,False,False,False,['Integrated Management of Childhood Illnesses...,0.910053,35.100313,
4,St Monicah Dispensary Mwengenye,St Monicah Dispensary Mwengenye,Level 2,Dispensary,Nairobi,Kasarani,Faith Based Organization,Not Admitting Patients,Operational,False,False,True,False,['Integrated Management of Childhood Illnesses...,-1.252112,36.918752,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13533,Kangeta SCH,Kangeta Heaith Centre,Level 4,Primary care hospitals,Meru,Igembe Central,Ministry of Health,,Operational,True,False,False,False,[],0.276440,37.889620,
13534,Kyuasini Health Centre,Kyuasini Health Centre,Level 3,Basic Health Centre,Makueni,Kaiti,Ministry of Health,,Operational,False,False,True,True,['Integrated Management of Childhood Illnesses...,-1.790280,37.470740,
13535,Rota Health Centre,Rota Health Centre,Level 3,Basic Health Centre,Kisumu,Kisumu West,Ministry of Health,Not Admitting Patients,Operational,False,False,False,False,[],-0.092820,34.674900,
13536,Kiganjo Health Centre,Kiganjo Health Centre,Level 3,Basic Health Centre,Nyeri,Nyeri Central,Ministry of Health,,Operational,False,False,False,False,[],-0.381190,37.002770,


In [ ]:

tree2 = BallTree(np.deg2rad(df2[['lat', 'long']].values.astype(float)), metric='minkowski') #minkowski #haversine

long_q = 37.02479170999999 	
lat_q = -1.13481782999

distances2, indices2 = tree2.query(np.deg2rad(np.c_[lat_q, long_q]), k = 5)

r_km = 6371 # multiplier to convert to km (from unit distance)

for d, ind in zip(distances2, indices2):
    print(d, ind)
    for i,index in enumerate(ind):
        print(d[i])
        df2['distance'][index] = d[i] * r_km
        # print("\n\t{} with distance of approx {} kms".format(df2['name'][index], d[i] * r_km) )

print(indices2[0])
df2.iloc[indices2[0]]

[0.00027041 0.00028409 0.00033676 0.00033917 0.00036104] [ 4928  4153  5030 10344  5958]
0.0002704092739712876
0.0002840937583252286
0.00033675745921606917
0.000339173782245905
0.0003610440726906987
[ 4928  4153  5030 10344  5958]


c:\Users\david.njuguna\OneDrive - Amref Health Africa\Documents\DaudiDev\env37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,name,officialname,keph_level_name,facility_type_name,county_name,sub_county_name,owner_type_name,admission_status_name,operation_status_name,open_whole_day,open_public_holidays,open_weekends,open_late_night,service_names,lat,long,distance
4928,Sawa Sawa Health Services,Sawa Sawa Health Services,Level 2,Medical Clinic,Kiambu,Juja,Private Practice,,Operational,False,True,True,True,"['Short Term', 'Integrated Child Immunization'...",-1.124511,37.013224,1.722777
4153,Boskem Medical clinic,Boskem Medical clinic,Level 2,Medical Clinic,Kiambu,Juja,Private Practice,,Operational,False,True,True,True,"['Accident and Emergency casualty Services', '...",-1.141298,37.009860,1.809961
5030,Chaka Medical Clinic,Chaka Medical Clinic,Level 2,Medical Clinic,Kiambu,Juja,Private Practice,,Operational,False,True,True,True,"['Integrated Child Immunization', 'Postnatal c...",-1.152966,37.031344,2.145482
10344,Kalimoni Mission Hospital (Juja),Kalimoni Mission Hospital (Juja),Level 4,Primary care hospitals,Kiambu,Juja,Faith Based Organization,,Operational,True,False,True,False,[],-1.116050,37.019750,2.160876
5958,Elias Medical Clinic,Elias Medical Clinic,Level 2,Medical Clinic,Kiambu,Juja,Private Practice,,Operational,False,False,False,False,[],-1.123150,37.007710,2.300212
